# Prepare EEG data for training of machine-learning models
# DIfferent approached

In this notebook, the preprocessing for machine learning purposes is done. 
We will compare doing it with and without classes

+ Import data.
+ Apply filters (bandpass).
+ Detect potential bad channels and replace them by interpolation.
+ Detect potential bad epochs and remove them.
+ Select features

## Import packages & links

In [1]:
# Import packages
import os
import sys

sys.path.insert(0, os.path.dirname(os.getcwd()))

import numpy as np
import h5py
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import mne

sns.set_theme(style="whitegrid")
%matplotlib inline
# %matplotlib qt

In [2]:
import fnmatch
import warnings
import re
warnings.filterwarnings('ignore')

import helper_functions

In [3]:
from config_for_repro_prepro  import PATH_RAW_DATA, PATH_METADATA, PATH_DATA_PROCESSED_ML

In [4]:
# add overwrite step 
PATH_RAW_DATA= 'C:/Projects/EEG_explorer/Data'

## Search all *.cnt files and get paths, code, and age group

Below I will show the original approach inside the markdown cell, but then follows my approach with list comprehension:
    

    dir_names = {11: '11mnd mmn',
                 17: '17mnd mmn',
                 23: '23mnd mmn',
                 29: '29mnd mmn',
                 35: '35mnd mmn',
                 41: '41mnd mmn',
                 47: '47mnd mmn'}

    df_list = []

    for age_group, directory in dir_names.items(): # Go into every age group folder        
        dir_path = os.path.join(PATH_RAW_DATA, directory)
        file_names = os.listdir(dir_path)
        cnt_paths = [os.path.join(dir_path, file_name) for file_name in fnmatch.filter(file_names, "*.cnt")]
        cnt_files = [x.split('/')[-1][:-4] for x in cnt_paths]
        codes = [int(re.search(r'\d+', x).group()) for x in cnt_files]
        df = pd.DataFrame(list(zip(codes, cnt_paths, cnt_files)), columns =['code', 'cnt_path','cnt_file']) 

        df['age_group'] = age_group
        df_list.append(df)

    cnt_files = pd.concat(df_list)

In [6]:
dir_names = {11: '11mnd mmn',
             17: '17mnd mmn',
             23: '23mnd mmn',
             29: '29mnd mmn',
             35: '35mnd mmn',
             41: '41mnd mmn',
             47: '47mnd mmn'}
    
df_list = []

for age_group, directory in dir_names.items(): # Go into every age group folder        
    dir_path = os.path.join('C:/Projects/EEG_explorer/Data', directory)
    file_names = os.listdir(dir_path)
    
    cnt_paths = [os.path.join(dir_path, file_name) for file_name in fnmatch.filter(file_names, "*.cnt")]
    # list comprehension - creates a list of all cnt file names with thisdirectory
    cnt_files = [os.path.basename(x)[:-4] for x in cnt_paths]
    # list comprehension
    codes = [int(re.search(r'\d+', x).group()) for x in cnt_files]
    # takes number out of string (\d+ takes out digits)
    df = pd.DataFrame(list(zip(codes, cnt_paths, cnt_files)), columns=['code', 'cnt_path','cnt_file']) 
    
    df['age_group'] = age_group
    df_list.append(df)

cnt_files = pd.concat(df_list)

In [7]:
cnt_files.head()

code                                           cnt_path  \
0     1  C:/Projects/EEG_explorer/Data\11mnd mmn\001_11...   
1     3  C:/Projects/EEG_explorer/Data\11mnd mmn\003_11...   
2     4  C:/Projects/EEG_explorer/Data\11mnd mmn\004_11...   
3     5  C:/Projects/EEG_explorer/Data\11mnd mmn\005_11...   
4     7  C:/Projects/EEG_explorer/Data\11mnd mmn\007_11...   

                                           cnt_file  age_group  
0     001_11_jc_mmn36_wk_mmn25_wk_mmn47_wk_mmn58_wk         11  
1  003_11_jc_mmn36_slp_mmn25_wk_mmn47_slp_mmn58_slp         11  
2                                     004_11_mc_mmn         11  
3                                    005_11_jc_mmn2         11  
4                              007_11_jc_mmn2_36_wk         11

## Search for all age files and create DataFrame containing all ages per subject code

The below could also be done with list comprehension, but I know it is a lot less data, so I care less.

In [8]:

age_files = {11: 'ages_11mnths.txt',
             17: 'ages_17mnths.txt',
             23: 'ages_23mnths.txt',
             29: 'ages_29mnths.txt',
             35: 'ages_35mnths.txt',
             41: 'ages_41mnths.txt',
             47: 'ages_47mnths.txt'}

df_list = []

for age_group, age_file in age_files.items():
    df = pd.read_csv(os.path.join(PATH_METADATA, 'ages', age_file), sep="\t")
    df['age_group'] = age_group
    df_list.append(df)

age_df = pd.concat(df_list)

In [9]:
age_df

code  age_days  age_months  age_years  age_group
0      1       329   10.966667   0.913889         11
1      3       336   11.200000   0.933333         11
2      5       329   10.966667   0.913889         11
3      6       319   10.633333   0.886111         11
4      7       329   10.966667   0.913889         11
..   ...       ...         ...        ...        ...
74   731      1409   46.966667   3.913889         47
75   732      1412   47.066667   3.922222         47
76   733      1404   46.800000   3.900000         47
77   734      1405   46.833333   3.902778         47
78   735      1398   46.600000   3.883333         47

[1717 rows x 5 columns]

### Merge the .cnt files with the age information we have on the subjects

In [10]:
#merged_df = pd.merge(age_df, cnt_files, how='left', on=['age_group','code'])

In [11]:
merged_df = pd.merge(cnt_files, age_df, how='left', on=['age_group', 'code'])

In [12]:
merged_df.head()

code                                           cnt_path  \
0     1  C:/Projects/EEG_explorer/Data\11mnd mmn\001_11...   
1     3  C:/Projects/EEG_explorer/Data\11mnd mmn\003_11...   
2     4  C:/Projects/EEG_explorer/Data\11mnd mmn\004_11...   
3     5  C:/Projects/EEG_explorer/Data\11mnd mmn\005_11...   
4     7  C:/Projects/EEG_explorer/Data\11mnd mmn\007_11...   

                                           cnt_file  age_group  age_days  \
0     001_11_jc_mmn36_wk_mmn25_wk_mmn47_wk_mmn58_wk         11     329.0   
1  003_11_jc_mmn36_slp_mmn25_wk_mmn47_slp_mmn58_slp         11     336.0   
2                                     004_11_mc_mmn         11       NaN   
3                                    005_11_jc_mmn2         11     329.0   
4                              007_11_jc_mmn2_36_wk         11     329.0   

   age_months  age_years  
0   10.966667   0.913889  
1   11.200000   0.933333  
2         NaN        NaN  
3   10.966667   0.913889  
4   10.966667   0.913889

## Check files with no label 

In [17]:
merged_df.loc[merged_df['age_days'].isnull()]

code                                           cnt_path  \
2        4  C:/Projects/EEG_explorer/Data\11mnd mmn\004_11...   
9        9  C:/Projects/EEG_explorer/Data\11mnd mmn\009_11...   
10       9  C:/Projects/EEG_explorer/Data\11mnd mmn\009_11...   
35      31  C:/Projects/EEG_explorer/Data\11mnd mmn\031_11...   
49     101  C:/Projects/EEG_explorer/Data\11mnd mmn\101_11...   
...    ...                                                ...   
1978   433  C:/Projects/EEG_explorer/Data\41mnd mmn\433_41...   
1979   433  C:/Projects/EEG_explorer/Data\41mnd mmn\433_41...   
2031   616  C:/Projects/EEG_explorer/Data\41mnd mmn\616-24...   
2123   119  C:/Projects/EEG_explorer/Data\47mnd mmn\119_47...   
2127   126  C:/Projects/EEG_explorer/Data\47mnd mmn\126_47...   

                         cnt_file  age_group  age_days  age_months  age_years  
2                   004_11_mc_mmn         11       NaN         NaN        NaN  
9     009_11_jc_mmn36_wk_mmn25_wk         11       NaN         NaN        NaN  
10             009_11_jc_mmn47_wk         11       NaN         NaN        NaN  
35             031_11_mc_mmn36_wk         11       NaN         NaN        NaN  
49                  101_11_jr_mmn         11       NaN         NaN        NaN  
...                           ...        ...       ...         ...        ...  
1978          433_41_jd_mmn25_slp         41       NaN         NaN        NaN  
1979          433_41_jd_mmn36_mix         41       NaN         NaN        NaN  
2031         616-240-41m-mc-mmn39         41       NaN         NaN        NaN  
2123              119_47_jr_mmn39         47       NaN         NaN        NaN  
2127              126_47_md_mmn36         47       NaN         NaN        NaN  

[73 rows x 7 columns]

### Fill in the missing age data based on the age group the subject is in

In [18]:
merged_df['age_months'].fillna(merged_df['age_group'], inplace=True)
merged_df['age_days'].fillna(merged_df['age_group']*30, inplace=True)
merged_df['age_years'].fillna(merged_df['age_group']/12, inplace=True)

## Custom cnt-file import function:

In [ ]:
The below cell is not indented correctly, but I'm keeping it to show the original approach.

In [6]:
# def read_cnt_file_b(file,
#                   label_group,
#                   event_idx = [2, 3, 4, 5, 12, 13, 14, 15],
#                   channel_set = "30",
#                   tmin = -0.2,
#                   tmax = 0.8,
#                   lpass = 0.5, 
#                   hpass = 40, 
#                   threshold = 5, 
#                   max_bad_fraction = 0.2,
#                   max_bad_channels = 2):
#     """ Function to read cnt file. Run bandpass filter. 
#     Then detect and correct/remove bad channels and bad epochs.
#     Store resulting epochs as arrays.
    
#     Args:
#     --------
#     file: str
#         Name of file to import.
#     label_group: int
#         Unique ID of specific group (must be >0).
#     channel_set: str
#         Select among pre-defined channel sets. Here: "30" or "62"
#     """
    
#     if channel_set == "30":
#         channel_set = ['O2', 'O1', 'OZ', 'PZ', 'P4', 'CP4', 'P8', 'C4', 'TP8', 'T8', 'P7', 
#                        'P3', 'CP3', 'CPZ', 'CZ', 'FC4', 'FT8', 'TP7', 'C3', 'FCZ', 'FZ', 
#                        'F4', 'F8', 'T7', 'FT7', 'FC3', 'F3', 'FP2', 'F7', 'FP1']
#     elif channel_set == "62":
#         channel_set = ['O2', 'O1', 'OZ', 'PZ', 'P4', 'CP4', 'P8', 'C4', 'TP8', 'T8', 'P7', 
#                        'P3', 'CP3', 'CPZ', 'CZ', 'FC4', 'FT8', 'TP7', 'C3', 'FCZ', 'FZ', 
#                        'F4', 'F8', 'T7', 'FT7', 'FC3', 'F3', 'FP2', 'F7', 'FP1', 'AFZ', 'PO3', 
#                        'P1', 'POZ', 'P2', 'PO4', 'CP2', 'P6', 'M1', 'CP6', 'C6', 'PO8', 'PO7', 
#                        'P5', 'CP5', 'CP1', 'C1', 'C2', 'FC2', 'FC6', 'C5', 'FC1', 'F2', 'F6', 
#                        'FC5', 'F1', 'AF4', 'AF8', 'F5', 'AF7', 'AF3', 'FPZ']
#     else:
#         print("Predefined channel set given by 'channel_set' not known...")
        
    
#     # Initialize array
#     signal_collection = np.zeros((0,len(channel_set),501))
#     label_collection = [] #np.zeros((0))
#     channel_names_collection = []
    
#     # Import file
#     try:
#         data_raw = mne.io.read_raw_cnt(file, eog='auto', preload=True, verbose=False)
#     except ValueError:
#         print("ValueError")
#         print("Could not load file:", file)
#         return None, None, None
    
#     # Band-pass filter (between 0.5 and 40 Hz. was 0.5 to 30Hz in Stober 2016)
#     data_raw.filter(0.5, 40, fir_design='firwin')

#     # Get events from annotations in the data
#     events_from_annot, event_dict = mne.events_from_annotations(data_raw)
    
#     # Set baseline:
#     baseline = (None, 0)  # means from the first instant to t = 0

#     # Select channels to exclude (if any)
#     channels_exclude = [x for x in data_raw.ch_names if x not in channel_set]
#     channels_exclude = [x for x in channels_exclude if x not in ['HEOG', 'VEOG']]
    
#     for event_id in event_idx:
#         if str(event_id) in event_dict:
#             # Pick EEG channels
#             picks = mne.pick_types(data_raw.info, meg=False, eeg=True, stim=False, eog=False,
#                                #exclude=data_exclude)#'bads'])
#                                    include=channel_set, exclude=channels_exclude)#'bads'])

#             epochs = mne.Epochs(data_raw, events=events_from_annot, event_id=event_dict,
#                                 tmin=tmin, tmax=tmax, proj=True, picks=picks,
#                                 baseline=baseline, preload=True, event_repeated='merge', verbose=False)

#             # Detect potential bad channels and epochs
#             bad_channels, bad_epochs = helper_functions.select_bad_epochs(epochs,
#                                                                           event_id,
#                                                                           threshold = threshold,
#                                                                           max_bad_fraction = max_bad_fraction)

#             # Interpolate bad channels
#             # ------------------------------------------------------------------
#             if len(bad_channels) > 0:
#                 if len(bad_channels) > max_bad_channels:
#                     print(20*'--')
#                     print("Found too many bad channels (" + str(len(bad_channels)) + ")")
#                     return None, None, None
#                 else:
#                     # MARK: Setting the montage is not verified yet (choice of standard montage)
#                     montage = mne.channels.make_standard_montage('standard_1020')
#                     montage.ch_names = [ch_name.upper() for ch_name in montage.ch_names]
#                     data_raw.set_montage(montage)
                    
#                     # TODO: Think about using all channels before removing (62 -> 30), to enable for better interpolation
                    
#                     # Mark bad channels:
#                     data_raw.info['bads'] = bad_channels
#                     # Pick EEG channels:
#                     picks = mne.pick_types(data_raw.info, meg=False, eeg=True, stim=False, eog=False,
#                                        #exclude=data_exclude)#'bads'])
#                                        include=channel_set, exclude=channels_exclude)#'bads'])
#                     epochs = mne.Epochs(data_raw, events=events_from_annot, event_id=event_dict,
#                                         tmin=tmin, tmax=tmax, proj=True, picks=picks,
#                                         baseline=baseline, preload=True, verbose=False)
                    
#                     # Interpolate bad channels using functionality of 'mne'
#                     epochs.interpolate_bads()
                    

#             # Get signals as array and add to total collection
#             channel_names_collection.append(epochs.ch_names)
#             signals_cleaned = epochs[str(event_id)].drop(bad_epochs).get_data()
#             signal_collection = np.concatenate((signal_collection, signals_cleaned), axis=0)
#             label_collection += [event_id + label_group] * signals_cleaned.shape[0]

#     return signal_collection, label_collection, channel_names_collection

In [7]:
def read_cnt_file(file,
                  label_group,
                  event_idx = [2, 3, 4, 5, 12, 13, 14, 15],
                  channel_set = "30",
                  tmin = -0.2,
                  tmax = 0.8,
                  lpass = 0.5, 
                  hpass = 40, 
                  threshold = 5, 
                  max_bad_fraction = 0.2,
                  max_bad_channels = 2):
    """ Function to read cnt file. Run bandpass filter. 
    Then detect and correct/remove bad channels and bad epochs.
    Store resulting epochs as arrays.
    
    Args:
    --------
    file: str
        Name of file to import.
    label_group: int
        Unique ID of specific group (must be >0).
    channel_set: str
        Select among pre-defined channel sets. Here: "30" or "62"
    """
    
    if channel_set == "30":
        channel_set = ['O2', 'O1', 'OZ', 'PZ', 'P4', 'CP4', 'P8', 'C4', 'TP8', 'T8', 'P7', 
                       'P3', 'CP3', 'CPZ', 'CZ', 'FC4', 'FT8', 'TP7', 'C3', 'FCZ', 'FZ', 
                       'F4', 'F8', 'T7', 'FT7', 'FC3', 'F3', 'FP2', 'F7', 'FP1']
    elif channel_set == "62":
        channel_set = ['O2', 'O1', 'OZ', 'PZ', 'P4', 'CP4', 'P8', 'C4', 'TP8', 'T8', 'P7', 
                       'P3', 'CP3', 'CPZ', 'CZ', 'FC4', 'FT8', 'TP7', 'C3', 'FCZ', 'FZ', 
                       'F4', 'F8', 'T7', 'FT7', 'FC3', 'F3', 'FP2', 'F7', 'FP1', 'AFZ', 'PO3', 
                       'P1', 'POZ', 'P2', 'PO4', 'CP2', 'P6', 'M1', 'CP6', 'C6', 'PO8', 'PO7', 
                       'P5', 'CP5', 'CP1', 'C1', 'C2', 'FC2', 'FC6', 'C5', 'FC1', 'F2', 'F6', 
                       'FC5', 'F1', 'AF4', 'AF8', 'F5', 'AF7', 'AF3', 'FPZ']
    else:
        print("Predefined channel set given by 'channel_set' not known...")
        
    
    # Initialize array
    signal_collection = np.zeros((0,len(channel_set),501))
    label_collection = [] #np.zeros((0))
    channel_names_collection = []
    
    # Import file
    try:
        data_raw = mne.io.read_raw_cnt(file, eog='auto', preload=True, verbose=False)
    except ValueError:
        print("ValueError")
        print("Could not load file:", file)
        return None, None, None
    
    # Band-pass filter (between 0.5 and 40 Hz. was 0.5 to 30Hz in Stober 2016)
    data_raw.filter(0.5, 40, fir_design='firwin')

    # Get events from annotations in the data
    events_from_annot, event_dict = mne.events_from_annotations(data_raw)
    
    # Set baseline:
    baseline = (None, 0)  # means from the first instant to t = 0

    # Select channels to exclude (if any)
    channels_exclude = [x for x in data_raw.ch_names if x not in channel_set]
    channels_exclude = [x for x in channels_exclude if x not in ['HEOG', 'VEOG']]
    
    for event_id in event_idx:
        if str(event_id) in event_dict:
            # Pick EEG channels
            picks = mne.pick_types(data_raw.info, meg=False, eeg=True, stim=False, eog=False,
                               #exclude=data_exclude)#'bads'])
                                   include=channel_set, exclude=channels_exclude)#'bads'])

            epochs = mne.Epochs(data_raw, events=events_from_annot, event_id=event_dict,
                                tmin=tmin, tmax=tmax, proj=True, picks=picks,
                                baseline=baseline, preload=True, event_repeated='merge', verbose=False)

            # Detect potential bad channels and epochs
            bad_channels, bad_epochs = helper_functions.select_bad_epochs(epochs,
                                                                          event_id,
                                                                          threshold = threshold,
                                                                          max_bad_fraction = max_bad_fraction)

            # Interpolate bad channels
            # ------------------------------------------------------------------
            if len(bad_channels) > 0:
                if len(bad_channels) > max_bad_channels:
                    print(20*'--')
                    print("Found too many bad channels (" + str(len(bad_channels)) + ")")
                    return None, None, None
                else:
                    # MARK: Setting the montage is not verified yet (choice of standard montage)
                    montage = mne.channels.make_standard_montage('standard_1020')
                    montage.ch_names = [ch_name.upper() for ch_name in montage.ch_names]
                    data_raw.set_montage(montage)
                    
                    # TODO: Think about using all channels before removing (62 -> 30), to enable for better interpolation
                    
                    # Mark bad channels:
                    data_raw.info['bads'] = bad_channels
                    # Pick EEG channels:
                    picks = mne.pick_types(data_raw.info, meg=False, eeg=True, stim=False, eog=False,
                                       #exclude=data_exclude)#'bads'])
                                       include=channel_set, exclude=channels_exclude)#'bads'])
                    epochs = mne.Epochs(data_raw, events=events_from_annot, event_id=event_dict,
                                        tmin=tmin, tmax=tmax, proj=True, picks=picks,
                                        baseline=baseline, preload=True, verbose=False)
                    
                    # Interpolate bad channels using functionality of 'mne'
                    epochs.interpolate_bads()
                    

            # Get signals as array and add to total collection
            channel_names_collection.append(epochs.ch_names)
            signals_cleaned = epochs[str(event_id)].drop(bad_epochs).get_data()
            signal_collection = np.concatenate((signal_collection, signals_cleaned), axis=0)
            label_collection += [event_id + label_group] * signals_cleaned.shape[0]

    return signal_collection, label_collection, channel_names_collection

In [8]:
signal_collect, label_collect, channel_names_collection = read_cnt_file(merged_df['cnt_path'][20], 
                                                                        merged_df['age_months'][0],
                                                                        event_idx = [2, 3, 4, 5, 12, 13, 14, 15],
                                                                        channel_set = "30",
                                                                        tmin = -0.2,
                                                                        tmax = 0.8,
                                                                        lpass = 0.5, 
                                                                        hpass = 40, 
                                                                        threshold = 5, 
                                                                        max_bad_fraction = 0.2,
                                                                        max_bad_channels = 2)

NameError: name 'merged_df' is not defined

In [50]:
signal_collect.shape

(1917, 30, 501)

In [51]:
len(label_collect)

1917

In [52]:
signal_collect.shape #, label_collect.shape

(1917, 30, 501)

In [53]:
# metadata_collection[:10]

# Feature extraction and save processed data

## Determine how to store the processed data

Below we're determining what the best method is to extract and save the features. At the end, we combine all the parts.

In [54]:
local_processed_data5 = 'C:/Projects/EEG_explorer/Data/data_processed_ML5'

Now we will only do ten files for the sake of memory, this is just a teaching notebook anyways.

In [56]:
for i, file in merged_df.head(10).iterrows():
    
    # Import data and labels
    signal_collect, label_collect, ch_names = read_cnt_file(file['cnt_path'],
                                                            file['age_months'],
                                                            event_idx = [2, 3, 4, 5, 12, 13, 14, 15],
                                                            channel_set = "30",
                                                            tmin = -0.2,
                                                            tmax = 0.8,
                                                            lpass = 0.5, 
                                                            hpass = 40, 
                                                            threshold = 5, 
                                                            max_bad_fraction = 0.2,
                                                            max_bad_channels = 2)    
        
    # Save data and labels
    # ---------------------------------------------------------
#     if signal_collect is None:
#         continue

    # Firstly, save the metadata which will be the same for all files derived from this .cnt file
    filename_metadata = os.path.join(local_processed_data4, "processed_data_" + file['cnt_file'] + ".csv")
    pd.DataFrame(file).transpose().to_csv(filename_metadata, sep=',', index=False, header=True)        
    
    # Save all the epochs to separate files
    filename_signal = os.path.join(local_processed_data4, "processed_data_" + file['cnt_file'] + ".npy")
    np.save(filename_signal, signal_collect)
    
    #break # MARK: Remove this!

Used Annotations descriptions: ['2', '3', '4', '5', '55', '66', '77', '88']
Found 247 bad epochs in a total of 30  channels.
Found bad channel (more than 72.0  bad epochs): Channel no:  9
Found bad channel (more than 72.0  bad epochs): Channel no:  4
Marked 129 bad epochs in a total of 360  epochs.
Found 220 bad epochs in a total of 27  channels.
Found bad channel (more than 72.0  bad epochs): Channel no:  9
Found bad channel (more than 72.0  bad epochs): Channel no:  4
Marked 66 bad epochs in a total of 360  epochs.
Found 311 bad epochs in a total of 30  channels.
Found bad channel (more than 72.0  bad epochs): Channel no:  4
Found bad channel (more than 72.0  bad epochs): Channel no:  9
Found bad channel (more than 72.0  bad epochs): Channel no:  8
Marked 200 bad epochs in a total of 360  epochs.
----------------------------------------
Found too many bad channels (3)
Used Annotations descriptions: ['0', '12', '13', '14', '15', '2', '3', '4', '5', '55', '66', '77', '88']
Found 110 ba

Found 23 bad epochs in a total of 4  channels.
Marked 23 bad epochs in a total of 400  epochs.
Found 44 bad epochs in a total of 8  channels.
Marked 44 bad epochs in a total of 400  epochs.
Found 11 bad epochs in a total of 18  channels.
Marked 11 bad epochs in a total of 400  epochs.
Found 2 bad epochs in a total of 2  channels.
Marked 2 bad epochs in a total of 50  epochs.
Found 1 bad epochs in a total of 2  channels.
Marked 1 bad epochs in a total of 50  epochs.
Found 3 bad epochs in a total of 6  channels.
Marked 3 bad epochs in a total of 50  epochs.
Found 1 bad epochs in a total of 1  channels.
Marked 1 bad epochs in a total of 50  epochs.
Used Annotations descriptions: ['0', '13', '3', '66']
Found 95 bad epochs in a total of 30  channels.
Marked 95 bad epochs in a total of 400  epochs.
Found 10 bad epochs in a total of 7  channels.
Marked 10 bad epochs in a total of 50  epochs.
Used Annotations descriptions: ['0', '12', '13', '2', '3', '55', '66']
Found 122 bad epochs in a total

Used Annotations descriptions: ['0', '12', '2', '55']
Found 14 bad epochs in a total of 29  channels.
Marked 14 bad epochs in a total of 400  epochs.
Found 2 bad epochs in a total of 22  channels.
Marked 2 bad epochs in a total of 50  epochs.
Used Annotations descriptions: ['12', '2', '55']
Found 386 bad epochs in a total of 2  channels.
Found bad channel (more than 80.0  bad epochs): Channel no:  8
Marked 48 bad epochs in a total of 400  epochs.
Found 50 bad epochs in a total of 2  channels.
Found bad channel (more than 10.0  bad epochs): Channel no:  8
Marked 3 bad epochs in a total of 50  epochs.
Used Annotations descriptions: ['0', '13', '3', '66']
Found 393 bad epochs in a total of 3  channels.
Found bad channel (more than 80.0  bad epochs): Channel no:  8
Found bad channel (more than 80.0  bad epochs): Channel no:  6
Marked 1 bad epochs in a total of 400  epochs.
Found 50 bad epochs in a total of 2  channels.
Found bad channel (more than 10.0  bad epochs): Channel no:  8
Found ba

Used Annotations descriptions: ['2', '3', '4', '5', '55', '66', '77', '80', '81', '88']
Found 20 bad epochs in a total of 4  channels.
Marked 20 bad epochs in a total of 450  epochs.
Found 8 bad epochs in a total of 4  channels.
Marked 8 bad epochs in a total of 450  epochs.
Found 5 bad epochs in a total of 1  channels.
Marked 5 bad epochs in a total of 450  epochs.
Found 108 bad epochs in a total of 30  channels.
Marked 108 bad epochs in a total of 450  epochs.
Used Annotations descriptions: ['2', '3', '4', '5', '55', '66', '77', '80', '81', '88']
Found 56 bad epochs in a total of 27  channels.
Marked 56 bad epochs in a total of 450  epochs.
Found 20 bad epochs in a total of 3  channels.
Marked 20 bad epochs in a total of 450  epochs.
Found 32 bad epochs in a total of 26  channels.
Marked 32 bad epochs in a total of 450  epochs.
Found 163 bad epochs in a total of 13  channels.
Found bad channel (more than 90.0  bad epochs): Channel no:  8
Marked 31 bad epochs in a total of 450  epochs

Found 164 bad epochs in a total of 29  channels.
Found bad channel (more than 80.0  bad epochs): Channel no:  1
Marked 115 bad epochs in a total of 400  epochs.
Found 190 bad epochs in a total of 30  channels.
Marked 190 bad epochs in a total of 400  epochs.
Found 202 bad epochs in a total of 30  channels.
Marked 202 bad epochs in a total of 400  epochs.
Found 25 bad epochs in a total of 17  channels.
Found bad channel (more than 10.0  bad epochs): Channel no:  1
Marked 16 bad epochs in a total of 50  epochs.
Found 21 bad epochs in a total of 28  channels.
Marked 21 bad epochs in a total of 50  epochs.
Found 28 bad epochs in a total of 27  channels.
Found bad channel (more than 10.0  bad epochs): Channel no:  8
Marked 27 bad epochs in a total of 50  epochs.
Used Annotations descriptions: ['0', '12', '13', '14', '15', '2', '3', '4', '5', '55', '66', '77', '88']
Found 565 bad epochs in a total of 28  channels.
Found bad channel (more than 160.0  bad epochs): Channel no:  8
Marked 198 bad

...

Used Annotations descriptions: ['13', '3', '99']
Found 2 bad epochs in a total of 3  channels.
Marked 2 bad epochs in a total of 999  epochs.
Found 1 bad epochs in a total of 1  channels.
Marked 1 bad epochs in a total of 150  epochs.
Used Annotations descriptions: ['13', '3', '99']
Found 89 bad epochs in a total of 30  channels.
Marked 89 bad epochs in a total of 1000  epochs.
Found 16 bad epochs in a total of 24  channels.
Marked 16 bad epochs in a total of 150  epochs.
Used Annotations descriptions: ['13', '3', '99']
Found 620 bad epochs in a total of 30  channels.
Found bad channel (more than 200.0  bad epochs): Channel no:  6
Marked 28 bad epochs in a total of 1000  epochs.
Found 96 bad epochs in a total of 30  channels.
Found bad channel (more than 30.0  bad epochs): Channel no:  6
Marked 6 bad epochs in a total of 150  epochs.
Used Annotations descriptions: ['13', '3', '99']
Found 8 bad epochs in a total of 4  channels.
Marked 8 bad epochs in a total of 1000  epochs.
No outliers

Used Annotations descriptions: ['13', '3', '99']
Found 48 bad epochs in a total of 30  channels.
Marked 48 bad epochs in a total of 607  epochs.
Found 8 bad epochs in a total of 9  channels.
Marked 8 bad epochs in a total of 150  epochs.
Used Annotations descriptions: ['12', '2', '55']
Found 61 bad epochs in a total of 14  channels.
Marked 61 bad epochs in a total of 800  epochs.
Found 9 bad epochs in a total of 7  channels.
Marked 9 bad epochs in a total of 100  epochs.
Used Annotations descriptions: ['13', '3', '66']
Found 17 bad epochs in a total of 6  channels.
Marked 17 bad epochs in a total of 800  epochs.
No outliers found with given threshold.
Used Annotations descriptions: ['13', '3', '66']
Found 80 bad epochs in a total of 29  channels.
Marked 80 bad epochs in a total of 875  epochs.
Found 21 bad epochs in a total of 9  channels.
Marked 21 bad epochs in a total of 109  epochs.
Used Annotations descriptions: ['13', '3', '66']
Found 19 bad epochs in a total of 8  channels.
Mark

Found 1 bad epochs in a total of 1  channels.
Marked 1 bad epochs in a total of 69  epochs.
Used Annotations descriptions: ['13', '3', '99']
Found 12 bad epochs in a total of 30  channels.
Marked 12 bad epochs in a total of 642  epochs.
Found 2 bad epochs in a total of 1  channels.
Marked 2 bad epochs in a total of 81  epochs.
Used Annotations descriptions: ['0', '13', '3', '99']
Found 35 bad epochs in a total of 9  channels.
Marked 35 bad epochs in a total of 1160  epochs.
Found 6 bad epochs in a total of 4  channels.
Marked 6 bad epochs in a total of 290  epochs.
Used Annotations descriptions: ['13', '3', '99']
Found 95 bad epochs in a total of 28  channels.
Marked 95 bad epochs in a total of 800  epochs.
Found 12 bad epochs in a total of 3  channels.
Marked 12 bad epochs in a total of 100  epochs.
Used Annotations descriptions: ['0', '13', '3', '99']
Found 4 bad epochs in a total of 3  channels.
Marked 4 bad epochs in a total of 840  epochs.
Found 2 bad epochs in a total of 2  chann

Used Annotations descriptions: ['13', '3', '66']
Found 18 bad epochs in a total of 5  channels.
Marked 18 bad epochs in a total of 800  epochs.
Found 1 bad epochs in a total of 1  channels.
Marked 1 bad epochs in a total of 100  epochs.
Used Annotations descriptions: ['13', '3', '66']
Found 6 bad epochs in a total of 9  channels.
Marked 6 bad epochs in a total of 800  epochs.
Found 1 bad epochs in a total of 1  channels.
Marked 1 bad epochs in a total of 100  epochs.
Used Annotations descriptions: ['13', '3', '66']
Found 19 bad epochs in a total of 30  channels.
Marked 19 bad epochs in a total of 800  epochs.
Found 3 bad epochs in a total of 2  channels.
Marked 3 bad epochs in a total of 100  epochs.
Used Annotations descriptions: ['13', '3', '66']
Found 44 bad epochs in a total of 17  channels.
Marked 44 bad epochs in a total of 800  epochs.
Found 11 bad epochs in a total of 6  channels.
Marked 11 bad epochs in a total of 100  epochs.
Used Annotations descriptions: ['13', '3', '66']
F

Used Annotations descriptions: ['13', '3', '66']
Found 58 bad epochs in a total of 14  channels.
Marked 58 bad epochs in a total of 800  epochs.
Found 8 bad epochs in a total of 6  channels.
Marked 8 bad epochs in a total of 100  epochs.
Used Annotations descriptions: ['13', '3', '66']
Found 56 bad epochs in a total of 30  channels.
Marked 56 bad epochs in a total of 1038  epochs.
Found 7 bad epochs in a total of 7  channels.
Marked 7 bad epochs in a total of 131  epochs.
Used Annotations descriptions: ['13', '3', '66']
Found 8 bad epochs in a total of 30  channels.
Marked 8 bad epochs in a total of 896  epochs.
Found 1 bad epochs in a total of 30  channels.
Marked 1 bad epochs in a total of 109  epochs.
ValueError
Could not load file: C:/Projects/EEG_explorer/Data\47mnd mmn\711-085-47m-jr-mmn36.cnt


OK, so much drama and so many page long functions....let's see how it might be approached differently:

In [ ]:
class FileReader:
    
    default_args = {
        'event_idx': [2, 3, 4, 5, 12, 13, 14, 15],
        'channel_set': '30',
        'tmin': -0.2,
        'tmax': 0.8,
        'lpass': 0.5, 
        'hpass': 40 ,
        'threshold': 5, 
        'max_bad_fraction': 0.2,
        'max_bad_channels': 2,
    }

    def read_cnt(self, row):
        return read_cnt_file(row['cnt_path'], row['age_months'], **self.default_args)
    
    def store_all(self, df, store_prefix, nrows=None):
        if nrows is None:
            rows = df.iterrows()
        else:
            rows = df.head(nrows).iterrows()

        for i, row in rows:
            signal_collect, label_collect, ch_names = self.read_cnt(row)

            if signal_collect is None:
                continue

            store_path = os.path.join(store_prefix, 'processed_data_' + row['cnt_file'])
            store_csv = store_path + '.csv'
            store_npy = store_path + '.npy'

            pd.DataFrame(row).transpose().to_csv(store_csv, sep=',', index=False, header=True)        
            np.save(store_npy, signal_collect)

Don't run the cell below, ask yourself- do I know how to make it do only 10? then change it to do so. (This is a learning exercise after all)

In [ ]:
FileReader().store_all(merged_df, local_processed_data5)

In the real world the class could be hidden in the library, and just the cell above should make it to the notebook.